# DAT 19: Homework 4 Assignment - SVMs, Trees, RF

## Instructions

The goal of this homework is to review and bring together what we have learned about Support Vector Machines, Decision Trees, Random Forests, and ensembles. 

Please do all your analysis to answer the questions below in this Jupyter notebook. Show your work.

**Please submit your completed notebook by 6:30PM on Wednesday, February 17.**

## About the Data

Use the cancer_uci.csv dataset in the Data directory of our course repo. This is the [Breast Cancer Wisconsin](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)) dataset from the UCI ML Repository.

## Homework Assignment

**1) Load the data and check for balance between the two classes. If the ratio is less than 60/40 rebalance the classes to 50/50. We've provided some help here to get you started.**

In [66]:
import pandas as pd
canc = pd.read_csv("../data/cancer_uci.csv", index_col=0)
canc.head()
canc.Class.value_counts()
# What are the frequencies of each class?

Benign       458
Malignant    241
Name: Class, dtype: int64

We have imbalanced classes so we need to decide if we want to undersample, and take only 241 values from the Benign category, or oversample, and artificially inflate the volume of malignant data. First, let's convert to binary 1,0 for classification.

In [67]:
canc.Class = canc.Class.map({'Benign':0,'Malignant':1})
canc.Class.value_counts()

0    458
1    241
Name: Class, dtype: int64

To undersample, we would throw away almost half of our benign examples, which would greatly alter our dataset and we don't want to lose that much info. So let's oversample! Here is a pattern for how to oversample:

In [68]:
# Separate your two classes:
mal_example = canc[canc.Class == 1]
benign_example = canc[canc.Class == 0]

# Oversample the malignant class to have a 50/50 ratio:
mal_over_example = mal_example.sample(458,replace=True)

# Recombine the two frames:
over_sample = pd.concat([mal_over_example,benign_example])

# Sanity check the length:
print len(over_sample)

916


In [69]:
over_sample.info()

#'Bare_Nuclei' has 17 instances of non-numerical '?' 
over_sample['Bare_Nuclei'].value_counts()

#Not sure if should drop or convert to mean... Gonna drop them
over_sample = over_sample[over_sample['Bare_Nuclei'] != '?']
over_sample['Bare_Nuclei'] = over_sample['Bare_Nuclei'].astype(float)
over_sample.info()


#over_sample['Bare_Nuclei'].replace(to_replace='nan')
#over_sample['Bare_Nuclei'] = over_sample['Bare_Nuclei'].fillna(over_sample['Bare_Nuclei'].mean())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 916 entries, 104 to 695
Data columns (total 11 columns):
Sample_code_number             916 non-null int64
Clump_Thickness                916 non-null int64
Uniformity_of_Cell_Size        916 non-null int64
Uniformity_of_Cell_Shape       916 non-null int64
Marginal_Adhesion              916 non-null int64
Single_Epithelial_Cell_Size    916 non-null int64
Bare_Nuclei                    916 non-null object
Bland_Chromatin                916 non-null int64
Normal_Nucleoli                916 non-null int64
Mitoses                        916 non-null int64
Class                          916 non-null int64
dtypes: int64(10), object(1)
memory usage: 85.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 896 entries, 104 to 695
Data columns (total 11 columns):
Sample_code_number             896 non-null int64
Clump_Thickness                896 non-null int64
Uniformity_of_Cell_Size        896 non-null int64
Uniformity_of_Cell_Shape       89

**2) Are the features normalized? If not, use the scikit-learn standard scaler to normalize them.**

In [ ]:
#your code here

**3) Train a linear SVM, using the cross validated accuracy as the score (use the scikit-learn method).**

In [57]:
from sklearn import svm
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict

bc_features = over_sample.drop(['Class','Sample_code_number'], axis=1)
bc_target = over_sample['Class']

X_train, X_test, y_train, y_test = train_test_split(bc_features, bc_target, test_size=.20, random_state=0)


svc_classifier = svm.SVC(kernel='linear', C=1)
svc_clf_fit = svc_classifier.fit(X_train,y_train)
predictions = svc_clf_fit.predict(X_test)


cross_val_scores = cross_val_score(svc_classifier, bc_features, bc_target, cv=5)
cross_val_scores

cv_accuracy_score = cross_val_scores.mean()
cv_accuracy_score

0.97218476903870155

**4) Display the confusion matrix, classification report, and AUC.**

In [59]:
#your code here
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[90  2]
 [ 2 86]]
             precision    recall  f1-score   support

          0       0.98      0.98      0.98        92
          1       0.98      0.98      0.98        88

avg / total       0.98      0.98      0.98       180



**5) Repeat steps 2 through 4 using a Decision Tree model. Are the results better or worse than the SVM?**

In [79]:
#your code here
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth=3, random_state=1)
tree_clf_fit = tree_clf.fit(X_train, y_train)
tree_predictions = tree_clf_fit.predict(X_test)

tree_cross_val_accuracy = cross_val_score(tree_clf, bc_features, bc_target, cv=5).mean()
tree_cross_val_accuracy

0.96438202247191018

In [80]:
print(confusion_matrix(y_test,tree_predictions))
print(classification_report(y_test,tree_predictions))

[[88  4]
 [ 3 85]]
             precision    recall  f1-score   support

          0       0.97      0.96      0.96        92
          1       0.96      0.97      0.96        88

avg / total       0.96      0.96      0.96       180



**6) Repeat steps 2 through 4 using a Random Forest model. Are the results better or worse than the SVM?**

In [78]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=5, n_estimators=100, max_features=3)
rf_clf_fit = rf_clf.fit(X_train,y_train)

rf_predictions = rf_clf_fit.predict(X_test)

rf_cross_val_accuracy = cross_val_score(rf_clf, bc_features, bc_target, cv=5).mean()
rf_cross_val_accuracy


0.97886392009987522

In [81]:
print(confusion_matrix(y_test,rf_predictions))
print(classification_report(y_test,rf_predictions))

[[90  2]
 [ 1 87]]
             precision    recall  f1-score   support

          0       0.99      0.98      0.98        92
          1       0.98      0.99      0.98        88

avg / total       0.98      0.98      0.98       180



### Extra Credit Questions
**The following questions are strongly encouraged, but not required for this homework assignment.**

**7) Combine the SVM and the Decision Tree model using the [Voting Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html). Are the results better than either of these base classifiers alone?**

In [ ]:
#your code here

**8) Train an SVM using the RBF kernel. Is this model better or worse?**

In [ ]:
#your code here